In [ ]:
import os
import mlflow.sklearn
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from mlflow import MlflowClient
from sklearn.ensemble import RandomForestRegressor



In [ ]:
client = MlflowClient()

In [ ]:
# Register model name in the model registry
name = "RandomForestRegression_app" 

registered_model = client.create_registered_model(name)

In [ ]:
domino_project_id = os.environ['DOMINO_PROJECT_ID']

In [ ]:
# create an experiment run in MLflow and register a model version
#Domino Experiments names are required to be unique globally. Hence adding the project id
mlflow.set_experiment(f'download-model-{domino_project_id}')
params = {"n_estimators": 3, "random_state": 42}
rfr = RandomForestRegressor(**params).fit([[0, 1]], [1])
# Log MLflow entities
with mlflow.start_run() as run:
    mlflow.log_params(params)
    model_info = mlflow.sklearn.log_model(rfr, artifact_path="sklearn-model")
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = "A testing version of the model"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))


In [ ]:
print("You created your first registered model!")
print(f"Go to the Models UI and click on the {name} model to see the Model Card.")


In [ ]:
directory_path = os.getcwd()+'/artifacts'

os.makedirs(directory_path, exist_ok=True)

In [ ]:
artifact_path = "sklearn-model"
model_name = mv.name
model_version = mv.version
client = MlflowClient()
mv = client.get_model_version(model_name, model_version)

dst_path = f"{directory_path}/models/{model_name}/v{model_version}"
downloaded_artifacts_path = mlflow.artifacts.download_artifacts(artifact_uri=mv.source, dst_path=dst_path)

print(mv.run_id)
print(mlflow.pyfunc.get_model_dependencies(mv.source))
##Browse the folder for the model
print(dst_path)

In [ ]:
loaded_model = mlflow.pyfunc.load_model(f"{dst_path}/sklearn-model")

In [ ]:
#Make a prediction
loaded_model.predict([[20,0]])

## Use Models from Model Registry in the App

To use the model in the App Commit this workspace and add the `artifacts` folder to your repo. The App can then simply load the model from there and use it

   ```
   loaded_model = mlflow.pyfunc.load_model(f"{dst_path}/sklearn-model")
   loaded_model.predict([[20,0]])
   ```



In [ ]:
#Using Run Id to download artifacts
run_id='722499c8b7d74ff1b85677b4554dfa2e'
artifact_path = "sklearn-model"
dst_path = f"{directory_path}/runs/{run_id}"
downloaded_artifacts_path = mlflow.artifacts.download_artifacts(run_id=run_id, 
                                                                artifact_path=artifact_path,
                                                                dst_path=dst_path)

print(f"Downloaded Artifacts Path = {downloaded_artifacts_path}")
loaded_model = mlflow.pyfunc.load_model(f"{dst_path}/sklearn-model")
loaded_model.predict([[20,0]])

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace *.ipynb